In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from cv2 import aruco
from cv2 import VideoCapture

class CameraIntrinsics:
    def __init__(self, matrix, distorsion):
        self.matrix = matrix
        self.distorsion = distorsion
    def from_file(self, matlab_calib_file):
        with open(matlab_calib_file, 'r') as f:
            lines = f.readlines()
            self.matrix = np.array([[float(x) for x in line.split()] for line in lines[2:6]])
            self.distorsion = np.array([[float(x) for x in line.split()] for line in lines[7:8]]

def track_through_frames(imgs: list, pattern=aruco.DICT_5X5_250, camera_intrinsics=CameraIntrinsics(None, None)) -> list:
    results = []
    for img in imgs:
        res, _, _ = aruco_single(img, pattern, camera_intrinsics)
        results.append(res)
    return results


def aruco_single(img, pattern=aruco.DICT_5X5_250, camera_intrinsics=CameraIntrinsics(None, None)):
    corners, ids, rejectedImgPoints = aruco.detectMarkers(
        img, 
        aruco.Dictionary_get(pattern),
        parameters=aruco.DetectorParameters_create(),
        matrix=camera_intrinsics.matrix,
        distCoeff=camera_intrinsics.distorsion
    )
    if ids is not None:
        rvecs, tvecs, _ = aruco.estimatePoseSingleMarkers(corners, 0.05, camera_intrinsics.matrix, camera_intrinsics.distorsion)
        for i in range(len(ids)):
            img = aruco.drawAxis(img, camera_intrinsics.matrix, camera_intrinsics.distorsion, rvecs[i], tvecs[i], 0.1)
    return img, corners, ids
